In [ ]:
from google.colab import drive
drive.mount('/content/drive')
main_directory = '/content/drive/My Drive/My Tasks/Youtube/Emotion.csv'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv(main_directory,encoding = 'utf-8',sep=';')

df.head()

,id,text,emotion,lan,domain
0,2275,Hands down. You are good. I mean smart good.,joy,EN,funny_video
1,2283,Your are very good. keep it up,none,EN,funny_video
2,2292,Man!! I will never get used to these videos of...,surprise,EN,funny_video
3,2293,ডিসলাইক দেয়া 32,none,BN,funny_video
4,2298,ব্যাপক বিনোদন।,joy,BN,funny_video


In [ ]:
# Get indexes where name column has value john
indexNames = df[(df['lan'] == 'EN') | (df['lan'] == 'RN')].index

# Delete these row indexes from dataFrame
df = df.drop(indexNames)

In [ ]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline



In [ ]:

df = df[pd.notnull(df['emotion'])]
print(df.head(10))
print(df['text'].apply(lambda x: len(x.split(' '))).sum())

      id                                               text  ... lan       domain
3   2293                                    ডিসলাইক দেয়া 32  ...  BN  funny_video
4   2298                                     ব্যাপক বিনোদন।  ...  BN  funny_video
8   2312           মন খারাপ থাকলেই ঢুকে পরি আপনার চ্যানেলে।  ...  BN  funny_video
13  2421  Taslima Akter কন্টেন্ট ক্রিয়েটর রা এক একটা প্র...  ...  BN  funny_video
14  2423                                    জাতি জানতে চায়।  ...  BN  funny_video
20  2570                           নতুন কোন ভিডিও কবে আসবে?  ...  BN  funny_video
25  2621                ভাই , আর কতো অপেক্ষা ?? পরের ভিডিওর  ...  BN  funny_video
41  2880                          কাচি দিয়া জিপলাইন বানাইছে  ...  BN  funny_video
47  2984                            আপনাদের ক্রিয়িটিভিটি :D  ...  BN  funny_video
51  3047             ছিঃ,এভাবে অপমান নিজের দেশের মুভিকে,ছিঃ  ...  BN  funny_video

[10 rows x 5 columns]
13796


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['text'] = df['text'].apply(clean_text)


In [ ]:
df['text'].apply(lambda x: len(x.split(' '))).sum()

1236

In [ ]:
X = df.text
y = df.emotion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.2980132450331126


In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)


y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.2947019867549669


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        62
     disgust       0.29      0.98      0.44        85
        fear       0.00      0.00      0.00         1
         joy       0.71      0.08      0.15        61
        none       0.20      0.02      0.03        61
         sad       0.00      0.00      0.00        12
    surprise       0.00      0.00      0.00        20

    accuracy                           0.29       302
   macro avg       0.17      0.15      0.09       302
weighted avg       0.27      0.29      0.16       302



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
